In [1]:
import pickle
import numpy as np
import scipy.stats as stats

In [2]:
# 우선 데이터를 가져오는 부분부터 설정
numlns = 6
chNum = 40
dam = 128 * 63

In [3]:

wholeData = np.zeros((32, 40, dam, chNum))
y_train = np.zeros((32, 40, 4))
numIns = 1
for sub in range(32):
    if sub < 9:
        fName = 'data/DEAP/s0'+str(sub+1)+'.dat'
    else:
        fName = 'data/DEAP/s'+str(sub+1)+'.dat'
    x = pickle.load(open(fName, 'rb'), encoding='latin1')
    data = x['data']
    labels_tmp = x['labels']
    
    labels = np.zeros((40,4))
    mode = 0
    if(mode ==0):
        for i in range(40):
            for j in range(4):
                if(labels_tmp[i,j] < 5):
                    labels[i,j] = 0
                else:
                    labels[i,j] = 1
    else:
        for i in range(40):
            for j in range(4):
                if(labels_tmp[i,j] < 3):
                    labels[i,j] = 0
                elif(labels_tmp[i,j] >= 3 and labels_tmp[i,j] < 6):
                    labels[i,j] = 1
                else:
                    labels[i,j] = 2

    labels_tmp = labels
    labels = np.zeros((40,4))
    for i in range(40):
        labels[i, :] = labels_tmp[i,:]

    data_tmp = np.zeros((40, 40, dam))
    for i in range(40):
        data_tmp[i,:,:]= data[i,:,0:0+dam]

    wholeData_sub = np.zeros((40, dam, chNum))
    for i in range(40):
        for j in range(chNum):
            wholeData_sub[i, :, j] = data_tmp[i, j, :]
            
    #print wholeData_sub.shape
    wholeData[sub, :,:,:] = wholeData_sub
    y_train[sub, :, :] = labels
    

In [4]:
def extract_data(target_data):
    target_res = []

    target_mean = target_data.mean(axis=2)
    target_median = np.median(target_data, axis=2)
    target_maximum = np.max(target_data, axis=2)
    target_minimum = np.min(target_data, axis=2)
    target_std = np.std(target_data, axis=2)
    target_var = np.var(target_data, axis=2)
    target_range = np.ptp(target_data, axis=2)
    target_skew = stats.skew(target_data, axis=2)
    target_kurtosis = stats.kurtosis(target_data, axis=2)
    
    return [target_mean, target_median, target_maximum, target_minimum, target_std, target_var, target_range, target_skew, target_kurtosis]

In [5]:
whole_extract_data = extract_data(wholeData)

In [6]:
print(whole_extract_data[0].shape)

(32, 40, 40)


In [7]:
print(wholeData.shape)

(32, 40, 8064, 40)


In [8]:
slide_extract_data = []

for c in range(0, 8064, 807):
    slide_extract_data.append(extract_data(wholeData[:, :, c: 807 + c, :]))

In [9]:
#이제 추출된 데이터를 내가 원하는 형식으로 만들어보자
processed_data = np.zeros((32, 40, 40, 101))
processed_data2 = np.zeros((32, 40, 4040))

for ps_num in range(32):
    for exp_num in range(40):
        #print('ps_num : {}, exp_num : {}'.format(ps_num, exp_num))
        processed_data[ps_num, exp_num, :, 0] = ps_num
        processed_data[ps_num, exp_num, :, 1] = exp_num
        
        for ch_num in range(40):

            for c, data in enumerate(whole_extract_data):
                processed_data[ps_num, exp_num, ch_num, 2 + c] = data[ps_num, exp_num, ch_num]
                
            for c, datas in enumerate(slide_extract_data):
                
                for d, data in enumerate(datas):
                    processed_data[ps_num, exp_num, ch_num, 11 + c * 9 + d] = data[ps_num, exp_num, ch_num]

In [10]:
print(processed_data.shape)
print(whole_extract_data[0].shape)
print(whole_extract_data[0][0][0][0])
print(y_train.shape)

(32, 40, 40, 101)
(32, 40, 40)
-0.028258582130643254
(32, 40, 4)


In [11]:
processed_data[30][25][39][1]

25.0

In [12]:
with open('y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('processed_data.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

In [13]:
print('data', processed_data.shape)
print('y_train', y_train.shape)

data (32, 40, 40, 101)
y_train (32, 40, 4)


In [14]:
zz = processed_data.shape

In [15]:
for c, data in enumerate(processed_data[0]):
    print(data.flatten().shape)

(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)
(4040,)


In [16]:
print(labels[:, :])
print(labels.shape)

[[1. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 0.]
 [1. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [0. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 1. 1.]
 [1. 0. 1. 0.]
 [0. 1. 1. 1.]
 [1. 1. 1. 0.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 0. 1.]
 [0. 1. 1. 1.]
 [1. 0. 1. 0.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]]
(40, 4)
